In [1]:
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import requests
from datetime import datetime

load_dotenv()
dbUrl = os.environ["ODOO_URL"]
dbName = os.environ["ODOO_NAME"]
dbUsername = os.environ["ODOO_USERNAME"]
dbPassword = os.environ["ODOO_PASSWORD"]
erAppId = os.environ["APP_ID"]

common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(dbUrl))
uid = common.authenticate(dbName, dbUsername, dbPassword, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(dbUrl))

# Data Handling Functions
def id_field_into_1value(jsonFile):
    for index, line in enumerate(jsonFile):
        for item in line:
            if type(line[item]) == list:
                jsonFile[index][item] = jsonFile[index][item][1]

df = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order',
    'search_read',
    [],
    {'fields': [
        "name",
        "create_date",
        "x_partner_en_sale_order",
        "partner_id",
        "user_id",
        "company_id",
        "country_id",
        "pricelist_id",
        "recurrence_id",
        "recurring_live",
        "x_studio_fecha_acordada_de_inicio_de_cobro_de_suscripcin",
        "end_date",
        "stage_id",
        "x_studio_piloto",
        "x_region_comercial",
        "x_studio_etapa",
        "note"
    ]}
)

id_field_into_1value(df)

df = pd.DataFrame(df)


# Create a dictionary for currency conversion
currency_mapping = {
    "ARS": "ARS",
    "CLP": "CLP",
    "CLF": "CLF",
    "PEN": "PEN",
    "COP": "COP",
    "MXN": "MXN",
    "USD": "USD",
    "BRL": "BRL",
    "CAD": "CAD",
    "CNY": "CNY",
    "EUR": "EUR",
    "GBP": "GBP",
    "UYU": "UYU",
    "PYG": "PYG",
    "BOB": "BOB",
    "CRC": "CRC",
    "DOP": "DOP"
}

# Construct the API URL ---> DESCARTADAS POR TENER BASE EN EUROS POR DEFAULT, NO SER GRATIS O NO TENER ALGUNAS DE LAS MONDEAS QUE REQUERIMOS
# url = f'http://api.exchangeratesapi.io/v1/latest?access_key={api_key}&symbols=USD,MXN,BRL,ARS,CLP,COP,PEN'
# url = f'http://data.fixer.io/api/latest?access_key={api_key}&base=USD&symbols=MXN,BRL,ARS,CLP,COP,PEN'
# url = f'https://api.currencybeacon.com/v1/latest?access_key={api_key}&symbols=MXN,BRL,ARS,CLP,COP,PEN'

# Send a GET request to the API
url = f'https://openexchangerates.org/api/latest.json?app_id={erAppId}'
headers = {"accept": "application/json"}

# manage api response
response = requests.get(url, headers=headers)
data = response.json()
# print(data)

# Extract exchange rates from the data
rates = data['rates']
# print(exchange_rates)

# map only the used currencies exchnage rates based on USD
usd_er = {currency_mapping[key]: rates[key] for key in currency_mapping.keys()}
# print(usd_er)

# Extract the currency from the 'pricelist_id' column
df['pricelist_id'] = df['pricelist_id'].str.extract(r'\((.*?)\)', expand=False)

# Map the full currency names to their three-letter ISO codes
df['pricelist_id'] = df['pricelist_id'].map(currency_mapping)

# Map the exchange rates to a new column based on 'pricelist_id'
df['exchange_rate'] = df['pricelist_id'].map(usd_er)

df = df.rename(columns={
    'id': 'order_id',
    'name': 'order_name',
    "partner_id": "customer_id",
    "x_studio_etapa": "etapa_del_pedido"
})

df3 = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order.stage',
    'search_read',
    [],
    {'fields': [
        "name",
        "category"
    ]}
)

id_field_into_1value(df3)

df3 = pd.DataFrame(df3)

df3 = df3.drop(columns=['id'])

df = df.merge(df3, how='left', left_on='stage_id', right_on='name')

df = df.drop(columns=['name'])

df2 = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'sale.order.line',
    'search_read',
    [],
    {'fields': [
        "order_id",
        "product_id",
        "product_uom_qty",
        "qty_delivered",
        "qty_delivered_method",
        "qty_invoiced",
        "price_unit",
        "discount",
        "currency_id",
        "price_subtotal",
        "recurring_monthly",
        "product_uom",
        "name",
        "project_id"
    ]}
)

id_field_into_1value(df2)

df2 = pd.DataFrame(df2)

df2 = df2.rename(columns={
    'id': 'line_id',
    "order_id": "order_name",
    "product_uom": "unidad_de_medida",
    "name": "descripcion",
    "qty_invoiced": "cantidad_facturada"
})

df4 = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'product.template',
    'search_read',
    [],
    {'fields': [
        "name",
        "recurring_invoice"
    ]}
)

id_field_into_1value(df4)

df4 = pd.DataFrame(df4)

df4 = df4.drop(columns=['id'])

df2['temp_join'] = df2['product_id'].str.replace(r' \(.*?\)', '', regex=True).str.strip()

df2 = df2.merge(df4, how='left', left_on='temp_join', right_on='name')

df2 = df2.drop(columns=['temp_join'])

df2 = df2.drop(columns=['name'])

# Left join df and df2 on the column order_id and order_name
export_df = df.merge(df2, how='left', left_on='order_name', right_on='order_name')

# Divide 'recurring_monthly' by 'exchange_rate' to convert to USD
export_df['recurring_monthly'] = round(export_df['recurring_monthly'].astype(float) / export_df['exchange_rate'], 2)

DATABASE_CONNECTION_URI = os.environ["DB_URL"]
# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

# insert the dataframe into the postgres table
export_df.to_sql('odoo_subscriptions', con=engine, if_exists='replace', index=False)


# Create DataFrame from the dictionary
# er_df = pd.DataFrame(list(usd_er.items()), columns=["currency", "exchange_rate"])

# Add a timestamp column with the current date and time
# er_df['timestamp'] = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

# Reorder columns for better readability
# er_df = er_df[['timestamp', 'currency', 'exchange_rate']].round({'exchange_rate': 2})


# Pivot the DataFrame to have currencies as columns
# er_df_pivoted = er_df.pivot(index='timestamp', columns='currency', values='exchange_rate').reset_index()

# Reorder columns for better readability
# er_df_pivoted = er_df_pivoted[['timestamp', 'USD', 'ARS', 'CLP', 'CLF', 'PEN', 'COP', 'MXN', 'BRL', 'EUR']]
# er_df_pivoted = er_df_pivoted[['timestamp', 'USD', 'ARS', 'CLP', 'CLF', 'PEN', 'COP', 'MXN', 'BRL', 'EUR'] + [col for col in er_df_pivoted.columns if col not in ['timestamp', 'USD', 'ARS', 'CLP', 'CLF', 'PEN', 'COP', 'MXN', 'BRL', 'EUR']]]


# And finally insert the dataframe into the postgres tableS
# er_df_pivoted.to_sql('usd_er', con=engine, if_exists='replace', index=False)
# er_df_pivoted.to_sql('usd_er', con=engine, if_exists='append', index=False)

229